<h2 align=center> Facial Expression Recognition</h2>

## Importing Libraries

In [1]:
!pip install utils
!pip install livelossplot
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
from livelossplot.tf_keras import PlotLossesCallback
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.9.1


## Getting total number of images of each category

In [2]:
for expression in os.listdir("train/train"):
    print(str(len(os.listdir("train/train/" + expression))) + " " + expression + " images")

150 Antonio Bruni images
237 Davide Bruni images
144 Davide Tonellotto images


## Generate Training and Validation Batches

In [3]:
img_size = 48
batch_size = 64

datagen_train = ImageDataGenerator(horizontal_flip=True)

train_generator = datagen_train.flow_from_directory("train/train/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_validation.flow_from_directory("test/test/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 531 images belonging to 3 classes.
Found 186 images belonging to 3 classes.


## Create CNN Model

In [4]:
# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(3, activation='softmax'))

opt = Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       2

C:\Users\david\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Visualize Model Architecture

## Training and Evaluating Model

In [5]:

%%time

epochs = 25
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
callbacks = [PlotLossesCallback(), checkpoint, reduce_lr]

history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = validation_steps
)

Epoch 1/25
8/8 [==============================] - 6s 577ms/step - loss: 0.6508 - accuracy: 0.7045 - val_loss: 33.3313 - val_accuracy: 0.0391
Epoch 2/25
8/8 [==============================] - 7s 831ms/step - loss: 0.2412 - accuracy: 0.9180 - val_loss: 56.4093 - val_accuracy: 0.0391
Epoch 3/25
8/8 [==============================] - 4s 518ms/step - loss: 0.1248 - accuracy: 0.9700 - val_loss: 56.2410 - val_accuracy: 0.0391
Epoch 4/25
8/8 [==============================] - 6s 818ms/step - loss: 0.0587 - accuracy: 0.9829 - val_loss: 49.7801 - val_accuracy: 0.0391
Epoch 5/25
8/8 [==============================] - 5s 619ms/step - loss: 0.0713 - accuracy: 0.9743 - val_loss: 35.7175 - val_accuracy: 0.0391
Epoch 6/25
8/8 [==============================] - 5s 649ms/step - loss: 0.0428 - accuracy: 0.9914 - val_loss: 25.6647 - val_accuracy: 0.0391
Epoch 7/25
8/8 [==============================] - 4s 518ms/step - loss: 0.0365 - accuracy: 0.9914 - val_loss: 20.3494 - val_accuracy: 0.0391
Epoch 8/25
8/

## Represent Model as JSON String

In [6]:
model_json = model.to_json()
model.save_weights('model_weights.h5')
with open("model.json", "w") as json_file:
    json_file.write(model_json)

## Class for loading model and weights --> ESEGUIRE DA QUI, PER TESTARE SENZA RIADDESTRARE

In [2]:
from tensorflow.keras.models import model_from_json
import numpy as np

import tensorflow as tf


class FacialExpressionModel(object):

    EMOTIONS_LIST = ["Antonio Bruni","Davide Bruni",
                    "Davide Tonellotto"]

    def __init__(self, model_json_file, model_weights_file):
        # load model from JSON file
        with open(model_json_file, "r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)

        # load weights into the new model
        self.loaded_model.load_weights(model_weights_file)
        self.loaded_model.make_predict_function()

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]


## Getting frames and doing prediction

In [3]:
  
import cv2
import numpy as np

facec = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
model = FacialExpressionModel("./model.json", "./model_weights.h5")
font = cv2.FONT_HERSHEY_SIMPLEX

class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(0)

    def __del__(self):
        self.video.release()

    # returns camera frames along with bounding boxes and predictions
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]

            roi = cv2.resize(fc, (48, 48))
            pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        return fr

## Function for showing output video

In [4]:
def gen(camera):
    while True:
        frame = camera.get_frame()
        cv2.imshow('Facial Expression Recognization',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

## Running the code

In [5]:
gen(VideoCamera())

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 33ms/step


In [11]:
''' from PIL import Image,ImageOps
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
folder = r'train\train\Davide Bruni'
count = 67
# count increase by 1 in each iteration
# iterate all files from a directory
for count in range(67,80):
    # Construct old file name
    source = folder +'\\'+ str(count)+".png"
    dest_m = folder +'\\'+ str(count)+"_m.png"
    dest_f = folder +'\\'+ str(count)+"_f.png"
    img = Image.open(source)
    img_g = img.convert('LA')
    img_g.save(source)
    img = Image.open(source)
    # img.save(source)
    im_mirror = ImageOps.mirror(img)
    im_mirror.save(dest_m)
    im_flip = ImageOps.flip(img)
    im_flip.save(dest_f)
'''

' from PIL import Image,ImageOps\nimport imageio\nimport imgaug as ia\nimport imgaug.augmenters as iaa\nfolder = r\'train\train\\Davide Bruni\'\ncount = 67\n# count increase by 1 in each iteration\n# iterate all files from a directory\nfor count in range(67,80):\n    # Construct old file name\n    source = folder +\'\\\'+ str(count)+".png"\n    dest_m = folder +\'\\\'+ str(count)+"_m.png"\n    dest_f = folder +\'\\\'+ str(count)+"_f.png"\n    img = Image.open(source)\n    img_g = img.convert(\'LA\')\n    img_g.save(source)\n    img = Image.open(source)\n    # img.save(source)\n    im_mirror = ImageOps.mirror(img)\n    im_mirror.save(dest_m)\n    im_flip = ImageOps.flip(img)\n    im_flip.save(dest_f)\n'